In [103]:
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime
import json
import re
import csv

In [3]:
RANKINGS_URL = "http://www.owgr.com/en/Ranking.aspx?pageNo={0}&pageSize=300&country=All"
PLAYER_URL = "http://www.owgr.com/en/Ranking/PlayerProfile.aspx?playerID={0}&year={1}"
PLAYER_HISTORY_URL = "http://www.owgr.com/layouts/OWGR/PlayerRankingsForGraph.aspx?playerID={0}"

In [3]:
def parse_ranking_table(soup):
    output = []
    table = soup.find('table')
    
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        col_idx = 0
        data = {}

        if len(cols) > 0:
            for col in cols:
                text = col.text.strip()
                if (text == '-'):
                    text = '0'

                if col_idx == 0:
                    data['current_week_rank'] = text
                elif col_idx == 1:
                    data['last_week_rank'] = text
                elif col_idx == 3:
                    img = col.find('img')
                    data['country'] = img['title']
                elif col_idx == 4:
                    data['player_name'] = text
                    u = col.find('a')
                    data['player_id'] = u['href'].rsplit('=')[1]
                elif col_idx == 5:
                    data['avg_points'] = text
                elif col_idx == 6:
                    data['total_points'] = text
                elif col_idx == 7:
                    data['events_played_divisor'] = text
                elif col_idx == 10:
                    data['events_played_all'] = text

                col_idx = col_idx + 1

            output.append(data)
            
    return output


def parse_ranking_week(soup):
    h2 = soup.find_all('section', attrs={'id': 'ranking_table'})[0].find_all('h2')
    t = soup.find_all('time', attrs={'class':'sub_header'})
    
    week_num = int(h2[0].text.replace('WEEK', '').strip())
    
    temp = t[0].text.strip().split(' ')
    temp[0] = temp[0][:-2] # remove th, nd, st
    week_date = datetime.strptime(str(temp[1]) + ' ' + str(temp[0]) + ' ' + str(temp[2]), '%B %d %Y')
    
    return week_num, week_date


def download_ranking_table(url):
    request = urllib.request.urlopen(url)
    html = request.read()
    soup = BeautifulSoup(html, 'html.parser')
    
    table = parse_ranking_table(soup)
    week_num, week_date = parse_ranking_week(soup)
    
    return week_num, week_date, table

In [4]:
rankings = []
for i in range(1, 31):
    url = RANKINGS_URL.replace('{0}', str(i))
    print('Downloading page', str(i))
    week_num, week_date, output = download_ranking_table(url)
    rankings = rankings + output
    
print('Total rankings downloaded', len(rankings))

8785

In [5]:
filepath = '../data/owgr-' + str(week_num) + '-' + str(week_date.year) + '{:02d}'.format(week_date.month) + '{:02d}'.format(week_date.day) + '.json'
with open(filepath, 'w') as f:
    json.dump(rankings, f)

In [90]:
def download_player_history(playerid):
    output = []
    url = PLAYER_HISTORY_URL.replace("{0}", str(playerid))
    
    request = urllib.request.urlopen(url)
    html = request.read()
    j = json.loads(html)
    
    week_pattern = '\d{4}-\d{2}-\d{2}'
    rank_pattern = '"rank":"\d*",'
    weeks = re.findall(week_pattern, j)
    ranks = re.findall(rank_pattern, j)
    
    for w, r in zip(weeks, ranks):
        #print('Week', w, '/ Rank', r.replace('"rank":"', "").replace('",', ''))
        output.append({'week': w, 'rank': int(r.replace('"rank":"', "").replace('",', ''))})
    
    return output

In [112]:
file = '../data/owgr-10-20190310.json'
output_file = '../data/owgr-history.csv'

with open(file, 'r') as f:
    data = json.load(f)

# write header to csv file
header = [['player_id', 'player_name', 'week_of', 'rank']]
with open(output_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(header)
    f.close()

# gather csv data for each player and write to file
for row in data:
    csv_data = []
    print('Downloading history for', row['player_name'], '/', str(row['player_id']))
    history = download_player_history(row['player_id'])
    
    for wk in history:
        item_row = []
        item_row.append(str(row['player_id']))
        item_row.append(row['player_name'])
        item_row.append(wk['week'])
        item_row.append(str(wk['rank']))
        csv_data.append(item_row)
        
    with open(output_file, 'a+', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(csv_data)
        f.close()
        
print('Done')

Done
